# Online Data Mining (ODM) 
## Competitive Benchmarking Tool (Bax-shop.nl vs Thomann, bol.com, MaxiAxi)

### Academic & Business Objective
This notebook implements a reproducible data-mining tool that:
1) Collects **product offering and pricing signals** (incl. discounts, availability, delivery promise) for selected categories,
2) Collects **retailer-level service propositions** (shipping/returns policies) and **expert support indicators**,
3) Stores outputs into **CSV** and a **relational database** (SQLite by default, easily extendable to Postgres).

The design aligns with the ODM module goal: **structured data acquisition** with relatively lightweight cleaning during/after scraping, enabling later statistical analysis.  
Sources: Module guide and assignment description. (See provided docs.)


In [1]:
# 0. Environment Setup


# Core
import re
import json
import time
import math
import hashlib
from dataclasses import dataclass, asdict
from datetime import datetime, timezone
from urllib.parse import urljoin, urlparse

import pandas as pd
import numpy as np

# Database
import sqlite3

# Scrapy
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

print("Setup OK. Timestamp:", datetime.now().isoformat())


Setup OK. Timestamp: 2026-01-19T14:11:05.785260


# 1. Ethical and Operational Controls (Rate limiting, Scope, Logging)

Web scraping must respect:
- website terms,
- `robots.txt`,
- rate limiting,
- and robust error handling.

This notebook implements:
- conservative concurrency,
- download delay,
- user-agent control,
- and structured logging fields in the dataset.

(ODM Lecture 1 & 2 content + module expectations.)


In [2]:
# 1. Global Configuration (MaxiAxi - Microfoons only)

from datetime import datetime

RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")

CONFIG = {
    "run_id": RUN_ID,
    "currency": "EUR",
    "max_pages_per_category": 20,       # adjust if needed
    "download_delay_s": 1.25,
    "concurrent_requests": 4,
    "user_agent": "AUAS-ODM-Scraper/1.0 (educational use)",
    "timeout_s": 25,
}

# Task seed (use the exact URL you provided)
MAXIAXI_MICROFOONS_URL = (
    "https://www.maxiaxi.com/microfoons/"
    "?_gl=1*wun1p3*_up*MQ..*_gs*MQ.."
    "&gclid=CjwKCAiAybfLBhAjEiwAI0mBBtBabnYALUttCuiFDxKEWjAqyPC4M-DxsOfcTrDui7s_I3pyu6CXMxoClCwQAvD_BwE"
    "&gbraid=0AAAAADo6YHPbIVtlWk2zNZUOX0tH2Wu3R"
)

# Only MaxiAxi
RETAILERS = {
    "maxiaxi": {
        "name": "MaxiAxi",
        "base_url": "https://www.maxiaxi.com/",
        "is_marketplace": False,
        "policy_urls": {
            "shipping_returns": "https://www.maxiaxi.com/klantenservice/",
        },
        "expert_support_urls": {
            "advice": "https://www.maxiaxi.com/advies/",
        },
        "category_seeds": {
            "microphones": MAXIAXI_MICROFOONS_URL,
        }
    }
}

# Only the selected task category
CATEGORIES = [
    {"category_id": 1, "category_name": "Microphones", "key": "microphones"},
]

print("Config loaded. Retailers:", list(RETAILERS.keys()), "Run:", RUN_ID)
print("Seed:", MAXIAXI_MICROFOONS_URL)


Config loaded. Retailers: ['maxiaxi'] Run: 20260119_141105
Seed: https://www.maxiaxi.com/microfoons/?_gl=1*wun1p3*_up*MQ..*_gs*MQ..&gclid=CjwKCAiAybfLBhAjEiwAI0mBBtBabnYALUttCuiFDxKEWjAqyPC4M-DxsOfcTrDui7s_I3pyu6CXMxoClCwQAvD_BwE&gbraid=0AAAAADo6YHPbIVtlWk2zNZUOX0tH2Wu3R


# 2. Data Model (ERD-aligned) — Minimal Working Subset

The full ERD in your report can be extensive; for the scraper tool we implement a *minimal, functional subset* that already supports:
- Retailer-level pages (policy/support)
- Product listing → product detail
- Offer observations (price, reference price, discount %, stock, delivery promise)
- Time-stamped snapshots (observed_at)

This notebook stores:
- `retailer`
- `category`
- `product_page` (retailer-specific product identity)
- `offer_observation` (time series)

This structure is aligned to the ERD principle: separate **product pages** from **time-stamped observations** and support later relational expansion.


In [3]:
import sqlite3
from pathlib import Path
from datetime import datetime, timezone

# 1. Run identifier

try:
    RUN_ID
except NameError:
    RUN_ID = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

# 2. Desktop path

DESKTOP_DIR = Path("/Users/feddekoster/Desktop")

if not DESKTOP_DIR.exists():
    raise OSError(
        f"Desktop directory not found at {DESKTOP_DIR}. "
        "Check that the username is correct or that Desktop exists."
    )

# Test write permissions explicitly
try:
    test_file = DESKTOP_DIR / ".odm_write_test"
    test_file.write_text("ok", encoding="utf-8")
    test_file.unlink()
except Exception as e:
    raise PermissionError(
        "Desktop is not writable by this Python/Jupyter process.\n\n"
        "macOS fix:\n"
        "System Settings → Privacy & Security → Files and Folders (or Full Disk Access)\n"
        "→ Enable Desktop access for the app running Jupyter (Terminal / VS Code / Anaconda).\n\n"
        f"Original error: {e}"
    )

# 3. Assignment output folder

OUT_DIR = DESKTOP_DIR / "ODM_Assignment2"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DB_PATH = OUT_DIR / f"odm_competitor_benchmark_{RUN_ID}.sqlite"


# 4. Database schema

DDL = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS retailer (
  retailer_id INTEGER PRIMARY KEY AUTOINCREMENT,
  retailer_key TEXT UNIQUE NOT NULL,
  name TEXT NOT NULL,
  base_url TEXT NOT NULL,
  is_marketplace INTEGER NOT NULL,
  created_at TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS category (
  category_id INTEGER PRIMARY KEY,
  category_key TEXT UNIQUE NOT NULL,
  category_name TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS product_page (
  product_page_id INTEGER PRIMARY KEY AUTOINCREMENT,
  retailer_id INTEGER NOT NULL,
  category_id INTEGER NOT NULL,
  url TEXT NOT NULL,
  retailer_product_id TEXT,
  page_title TEXT,
  brand TEXT,
  gtin_ean TEXT,
  last_seen_at TEXT NOT NULL,
  UNIQUE(retailer_id, url),
  FOREIGN KEY(retailer_id) REFERENCES retailer(retailer_id),
  FOREIGN KEY(category_id) REFERENCES category(category_id)
);

CREATE TABLE IF NOT EXISTS offer_observation (
  observation_id INTEGER PRIMARY KEY AUTOINCREMENT,
  product_page_id INTEGER NOT NULL,
  observed_at TEXT NOT NULL,
  price_current REAL,
  price_reference REAL,
  discount_pct REAL,
  promo_flag INTEGER,
  stock_text_raw TEXT,
  delivery_promise_text TEXT,
  currency TEXT NOT NULL,
  http_status INTEGER,
  scrape_run_id TEXT NOT NULL,
  FOREIGN KEY(product_page_id) REFERENCES product_page(product_page_id)
);

CREATE TABLE IF NOT EXISTS retailer_page_capture (
  capture_id INTEGER PRIMARY KEY AUTOINCREMENT,
  retailer_id INTEGER NOT NULL,
  page_type TEXT NOT NULL,
  source_url TEXT NOT NULL,
  captured_at TEXT NOT NULL,
  content_text TEXT,
  http_status INTEGER,
  scrape_run_id TEXT NOT NULL,
  FOREIGN KEY(retailer_id) REFERENCES retailer(retailer_id)
);
"""

# 5. Create database

def db_connect(path: Path):
    con = sqlite3.connect(str(path))
    con.execute("PRAGMA foreign_keys = ON;")
    return con

with db_connect(DB_PATH) as con:
    con.executescript(DDL)


In [4]:
def db_connect(path: Path = DB_PATH):
    con = sqlite3.connect(str(path))
    con.execute("PRAGMA foreign_keys = ON;")
    return con

In [5]:
from datetime import datetime, timezone

# 2b. Seed reference tables

def upsert_retailers(con):
    now = datetime.now(timezone.utc).isoformat()
    for r_key, r in RETAILERS.items():
        con.execute("""
            INSERT INTO retailer (retailer_key, name, base_url, is_marketplace, created_at)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(retailer_key) DO UPDATE SET
                name=excluded.name,
                base_url=excluded.base_url,
                is_marketplace=excluded.is_marketplace
        """, (r_key, r["name"], r["base_url"], int(r["is_marketplace"]), now))
    con.commit()

def seed_categories(con):
    for c in CATEGORIES:
        con.execute("""
            INSERT OR REPLACE INTO category (category_id, category_key, category_name)
            VALUES (?, ?, ?)
        """, (c["category_id"], c["key"], c["category_name"]))
    con.commit()

with db_connect() as con:  
    upsert_retailers(con)
    seed_categories(con)

In [6]:
from pathlib import Path

OUTPUT_DIR = OUT_DIR  
PROD_CSV_PATH = OUTPUT_DIR / f"product_observations_{RUN_ID}.csv"
RET_CSV_PATH  = OUTPUT_DIR / f"retailer_pages_{RUN_ID}.csv"

# 3. Helper Functions (Normalization, Discount Formula, Safe Text Extraction)

## Discount depth formula
If a reference/list price is available:

\[
\text{Discount \%} = \frac{\text{Reference price} - \text{Current price}}{\text{Reference price}} \times 100
\]

If reference price is missing or equals 0, discount is undefined and set to null.

We also normalize:
- currency symbols,
- thousand/decimal separators,
- whitespace.


In [7]:
# 3. Helper functions (Scraper 2.0 - MaxiAxi Microfoons)

import re
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

def clean_text(x):
    if x is None:
        return None
    x = str(x)
    x = re.sub(r"\s+", " ", x).strip()
    return x if x else None

def parse_price(price_text):
    """
    Parse price strings like:
    '€ 39,95' -> 39.95
    '39,95'   -> 39.95
    """
    if not price_text:
        return None
    t = clean_text(price_text)
    if not t:
        return None

    # Keep digits, comma, dot
    t = re.sub(r"[^\d,\.]", "", t)

    # If comma is used as decimal separator
    # Example: 39,95 -> 39.95
    if t.count(",") == 1 and t.count(".") == 0:
        t = t.replace(",", ".")
    # If both appear, assume dot is thousands and comma is decimal: 1.299,95 -> 1299.95
    elif t.count(",") == 1 and t.count(".") >= 1:
        t = t.replace(".", "").replace(",", ".")

    try:
        return float(t)
    except:
        return None

def calc_discount_pct(price_reference, price_current):
    if price_reference is None or price_current is None:
        return None
    if price_reference <= 0:
        return None
    if price_current >= price_reference:
        return 0.0
    return round((price_reference - price_current) / price_reference * 100.0, 2)

def strip_tracking(url: str) -> str:
    """
    Benchmark-inspired URL normaliser (based on your classmate’s bol_products.py).
    Removes common marketing/tracking parameters so URLs are stable across runs.
    """
    if not url:
        return url
    try:
        p = urlparse(url)
        q = parse_qs(p.query)

        drop_keys = {
            "gclid", "gbraid", "wbraid", "fbclid",
            "utm_source", "utm_medium", "utm_campaign", "utm_term", "utm_content",
            "_gl", "_ga", "_gid", "mc_cid", "mc_eid",
            "ref", "cid", "source"
        }

        for k in list(q.keys()):
            if k.lower() in drop_keys:
                q.pop(k, None)

        new_query = urlencode(q, doseq=True)
        return urlunparse((p.scheme, p.netloc, p.path, p.params, new_query, p.fragment))
    except Exception:
        return url

# 4. Scrapy Items (Structured Output)

We emit two item types:
1) `ProductObservationItem` — for product detail pages (offer observations).
2) `RetailerPageItem` — for policy/support pages (retailer-level capture).

This directly supports the assignment requirement: deliver both **code (system)** and **data (datasets)**.


In [8]:
# 4. Scrapy Items

class ProductObservationItem(scrapy.Item):
    retailer_key = scrapy.Field()
    category_key = scrapy.Field()
    product_url = scrapy.Field()
    page_title = scrapy.Field()
    retailer_product_id = scrapy.Field()
    brand = scrapy.Field()
    gtin_ean = scrapy.Field()

    price_current = scrapy.Field()
    price_reference = scrapy.Field()
    discount_pct = scrapy.Field()
    promo_flag = scrapy.Field()

    stock_text_raw = scrapy.Field()
    delivery_promise_text = scrapy.Field()

    observed_at = scrapy.Field()
    http_status = scrapy.Field()
    scrape_run_id = scrapy.Field()

class RetailerPageItem(scrapy.Item):
    retailer_key = scrapy.Field()
    page_type = scrapy.Field()     # 'policy' or 'expert_support'
    source_url = scrapy.Field()
    captured_at = scrapy.Field()
    content_text = scrapy.Field()
    http_status = scrapy.Field()
    scrape_run_id = scrapy.Field()

# 5. Spider Implementation (Generic Baseline)

This spider is **generic**: it:
- starts from category listing pages for each retailer,
- collects product detail URLs from listing pages,
- visits product pages to extract benchmark variables.

Important:
- Each retailer’s HTML differs; you must adapt selectors per domain.
- The notebook provides **clear selector placeholders** and a repeatable pattern.
- Start with 1 retailer + 1 category, validate output, then expand.

For dynamic pages:
- Convert `scrapy.Request` to `SeleniumRequest` and add waits/scripts when required.


In [9]:
# 5. Spider (Scraper 2.0 - MaxiAxi Microfoons)

import scrapy
from urllib.parse import urljoin
from datetime import datetime, timezone
import re

class CompetitorBenchmarkSpider(scrapy.Spider):
    name = "maxiaxi_microfoons"
    allowed_domains = ["www.maxiaxi.com", "maxiaxi.com"]

    custom_settings = {
        "USER_AGENT": CONFIG["user_agent"],
        "DOWNLOAD_DELAY": CONFIG["download_delay_s"],
        "CONCURRENT_REQUESTS": CONFIG["concurrent_requests"],
        "LOG_LEVEL": "INFO",
        "DOWNLOAD_TIMEOUT": CONFIG["timeout_s"],
        "ROBOTSTXT_OBEY": True,
        "ITEM_PIPELINES": {
            "__main__.SQLitePipeline": 300,
            "__main__.CSVPipeline": 400,
        }
    }

    def start_requests(self):
        r_key = "maxiaxi"
        r = RETAILERS[r_key]

        # Retailer-level pages (optional but useful for ERD completeness)
        for page_type, url in (r.get("policy_urls") or {}).items():
            yield scrapy.Request(
                url=strip_tracking(url),
                callback=self.parse_retailer_page,
                meta={"retailer_key": r_key, "page_type": page_type}
            )

        for page_type, url in (r.get("expert_support_urls") or {}).items():
            yield scrapy.Request(
                url=strip_tracking(url),
                callback=self.parse_retailer_page,
                meta={"retailer_key": r_key, "page_type": page_type}
            )

        # Category seed: microphones only
        seed = r["category_seeds"]["microphones"]
        yield scrapy.Request(
            url=strip_tracking(seed),
            callback=self.parse_listing,
            meta={"retailer_key": r_key, "category_key": "microphones", "page_no": 1}
        )

    def parse_retailer_page(self, response):
        retailer_key = response.meta["retailer_key"]
        page_type = response.meta["page_type"]

        body_text = clean_text(" ".join(response.css("body *::text").getall()))
        if body_text:
            body_text = body_text[:5000]

        yield RetailerPageItem(
            retailer_key=retailer_key,
            page_type=page_type,
            source_url=strip_tracking(response.url),
            captured_at=datetime.now(timezone.utc).isoformat(),
            content_text=body_text,
            http_status=response.status,
            scrape_run_id=RUN_ID
        )

    def parse_listing(self, response):
        retailer_key = response.meta["retailer_key"]
        category_key = response.meta["category_key"]
        page_no = response.meta.get("page_no", 1)

        self.logger.info("LISTING page=%s status=%s url=%s", page_no, response.status, response.url)

        raw_links = response.css(
            "ol.products li.product-item a.product-item-link::attr(href),"
            "a.product-item-link::attr(href)"
        ).getall()

        links = [strip_tracking(urljoin(response.url, h)) for h in raw_links if h]
        links = list(dict.fromkeys(links))

        def is_product_url(u: str) -> bool:
            if not u or "maxiaxi.com" not in u:
                return False

            low = u.lower()

            # Drop category itself and irrelevant areas
            if "/microfoons/" in low:
                return False
            if any(x in low for x in ["/klantenservice", "/advies", "/blog", "/account", "/checkout"]):
                return False

            # MaxiAxi product URLs often look like:
            # https://www.maxiaxi.com/<slug>/
            return bool(re.match(r"^https://www\.maxiaxi\.com/[^/]+/?$", u))

        product_links = [u for u in links if is_product_url(u)]

        self.logger.info("LISTING found_links=%s product_links=%s", len(links), len(product_links))

        for u in product_links:
            yield scrapy.Request(
                url=u,
                callback=self.parse_product,
                meta={"retailer_key": retailer_key, "category_key": category_key}
            )

        # Pagination
        if page_no < CONFIG["max_pages_per_category"]:
            next_href = response.css(
                "li.pages-item-next a::attr(href), a.action.next::attr(href)"
            ).get()

            if next_href:
                next_url = strip_tracking(urljoin(response.url, next_href))
                yield scrapy.Request(
                    url=next_url,
                    callback=self.parse_listing,
                    meta={"retailer_key": retailer_key, "category_key": category_key, "page_no": page_no + 1}
                )

    def parse_product(self, response):
        retailer_key = response.meta["retailer_key"]
        category_key = response.meta["category_key"]

        product_url = strip_tracking(response.url)

        page_title = clean_text(response.css("h1.page-title span::text, h1::text").get())
        if not page_title:
            page_title = clean_text(response.css("title::text").get())

        # Prices
        price_cur_raw = clean_text(response.css(
            "span.price-final_price span.price::text,"
            "span.special-price span.price::text,"
            "span.price-wrapper span.price::text"
        ).get() or "")

        price_ref_raw = clean_text(response.css(
            "span.old-price span.price::text,"
            "span.regular-price span.price::text"
        ).get() or "")

        price_current = parse_price(price_cur_raw)
        price_reference = parse_price(price_ref_raw)
        discount_pct = calc_discount_pct(price_reference, price_current)
        promo_flag = int(discount_pct is not None and discount_pct > 0)

        # Stock
        stock_text_raw = clean_text(" ".join(response.css(
            ".stock.available *::text, .stock.unavailable *::text, .availability *::text"
        ).getall()))

        # Delivery promise
        delivery_promise_text = clean_text(" ".join(response.xpath(
            "//*[contains(normalize-space(), 'Bestel voor') or contains(normalize-space(), 'morgen')]/text()"
        ).getall()))

        # Specs extraction
        def value_after_label(label: str):
            v = response.xpath(
                f"//th[normalize-space()='{label}']/following-sibling::td[1]//text()"
            ).getall()
            if v:
                return clean_text(" ".join(v))

            v2 = response.xpath(
                f"//*[normalize-space()='{label}']/following::*[1]//text()"
            ).getall()
            return clean_text(" ".join(v2)) if v2 else None

        brand = value_after_label("Merk")
        retailer_product_id = value_after_label("SKU")     # mapping SKU -> retailer_product_id
        gtin_ean = value_after_label("EAN Code")

        yield ProductObservationItem(
            retailer_key=retailer_key,
            category_key=category_key,
            product_url=product_url,
            page_title=page_title,
            retailer_product_id=retailer_product_id,
            brand=brand,
            gtin_ean=gtin_ean,

            price_current=price_current,
            price_reference=price_reference,
            discount_pct=discount_pct,
            promo_flag=promo_flag,

            stock_text_raw=stock_text_raw,
            delivery_promise_text=delivery_promise_text,

            observed_at=datetime.now(timezone.utc).isoformat(),
            http_status=response.status,
            scrape_run_id=RUN_ID
        )


# 6. Pipelines: Store into SQLite + CSV

Two outputs are required for the assignment:
- datasets (CSV),
- structured storage (database preferred).

We implement both:
- SQLite insertion (relational),
- CSV exports (data deliverable + transparency).


In [10]:
import pandas as pd

# 6. Pipelines


class SQLitePipeline:
    def open_spider(self, spider):
        self.con = db_connect()
        self.cur = self.con.cursor()

    def close_spider(self, spider):
        self.con.commit()
        self.con.close()

    def _get_retailer_id(self, retailer_key):
        row = self.cur.execute(
            "SELECT retailer_id FROM retailer WHERE retailer_key=?",
            (retailer_key,)
        ).fetchone()
        if not row:
            raise RuntimeError(
                f"Retailer '{retailer_key}' not found in DB. "
                "Did you run the seeding cell (upsert_retailers)?"
            )
        return row[0]

    def _get_category_id(self, category_key):
        row = self.cur.execute(
            "SELECT category_id FROM category WHERE category_key=?",
            (category_key,)
        ).fetchone()
        if not row:
            raise RuntimeError(
                f"Category '{category_key}' not found in DB. "
                "Did you run the seeding cell (seed_categories)?"
            )
        return row[0]

    def _upsert_product_page(self, retailer_id, category_id, item):
        now = datetime.now(timezone.utc).isoformat()
        self.cur.execute("""
            INSERT INTO product_page (
              retailer_id, category_id, url, retailer_product_id, page_title, brand, gtin_ean, last_seen_at
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(retailer_id, url) DO UPDATE SET
                retailer_product_id=excluded.retailer_product_id,
                page_title=excluded.page_title,
                brand=excluded.brand,
                gtin_ean=excluded.gtin_ean,
                last_seen_at=excluded.last_seen_at
        """, (
            retailer_id, category_id,
            item.get("product_url"),
            item.get("retailer_product_id"),
            item.get("page_title"),
            item.get("brand"),
            item.get("gtin_ean"),
            now
        ))
        row = self.cur.execute(
            "SELECT product_page_id FROM product_page WHERE retailer_id=? AND url=?",
            (retailer_id, item.get("product_url"))
        ).fetchone()
        return row[0]

    def process_item(self, item, spider):
        d = dict(item)

        # Retailer-level pages (policy/support)
        if isinstance(item, RetailerPageItem):
            retailer_id = self._get_retailer_id(d["retailer_key"])
            self.cur.execute("""
                INSERT INTO retailer_page_capture (
                  retailer_id, page_type, source_url, captured_at, content_text, http_status, scrape_run_id
                )
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                retailer_id,
                d["page_type"],
                d["source_url"],
                d["captured_at"],
                d.get("content_text"),
                d.get("http_status"),
                d["scrape_run_id"]
            ))
            self.con.commit()
            return item

        # Product observation (price/stock/delivery)
        if isinstance(item, ProductObservationItem):
            retailer_id = self._get_retailer_id(d["retailer_key"])
            category_id = self._get_category_id(d["category_key"])
            pp_id = self._upsert_product_page(retailer_id, category_id, d)

            self.cur.execute("""
                INSERT INTO offer_observation (
                  product_page_id, observed_at, price_current, price_reference, discount_pct,
                  promo_flag, stock_text_raw, delivery_promise_text, currency, http_status, scrape_run_id
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                pp_id,
                d["observed_at"],
                d.get("price_current"),
                d.get("price_reference"),
                d.get("discount_pct"),
                d.get("promo_flag"),
                d.get("stock_text_raw"),
                d.get("delivery_promise_text"),
                CONFIG["currency"],
                d.get("http_status"),
                d["scrape_run_id"]
            ))
            self.con.commit()
            return item

        return item


class CSVPipeline:
    def open_spider(self, spider):
        self.products = []
        self.pages = []

    def close_spider(self, spider):
        if self.products:
            dfp = pd.DataFrame(self.products)
            dfp.to_csv(PROD_CSV_PATH, index=False)

        if self.pages:
            dfr = pd.DataFrame(self.pages)
            dfr.to_csv(RET_CSV_PATH, index=False)

        print("CSV saved to:")
        if self.products:
            print("-", PROD_CSV_PATH)
        if self.pages:
            print("-", RET_CSV_PATH)

    def process_item(self, item, spider):
        if isinstance(item, ProductObservationItem):
            self.products.append(dict(item))
        elif isinstance(item, RetailerPageItem):
            self.pages.append(dict(item))
        return item


# 7. Run the Scraper

Important:
- Start small (1 retailer + 1 category) to validate selectors.
- Then expand categories/pages once extraction quality is correct.

If you see many irrelevant URLs in listing extraction:
- refine the listing selectors immediately (this is normal in early sprint).


In [11]:
# 7. Run Spider (ONE run method only)

from scrapy.crawler import CrawlerProcess

# IMPORTANT:
# Twisted reactor can only run once per Jupyter kernel.
# If you want to run the spider again:
# Kernel -> Restart Kernel -> Run all cells once

process = CrawlerProcess(settings={})
process.crawl(CompetitorBenchmarkSpider)
process.start()

print("Scrape finished. Outputs saved:")
print("-", PROD_CSV_PATH)
print("-", RET_CSV_PATH)
print("-", DB_PATH)


2026-01-19 14:11:05 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2026-01-19 14:11:05 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.13.8, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 24.11.0, Python 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ], pyOpenSSL 25.0.0 (OpenSSL 3.0.17 1 Jul 2025), cryptography 44.0.1, Platform macOS-15.6.1-arm64-arm-64bit-Mach-O
2026-01-19 14:11:05 [scrapy.addons] INFO: Enabled addons:
[]
2026-01-19 14:11:05 [scrapy.extensions.telnet] INFO: Telnet Password: 33dded98808e5787
2026-01-19 14:11:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2026-01-19 14:11:05 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS': 4,
 'DOWNLOAD_DELAY': 1.25,
 'DOWNLOAD_TIMEOUT': 25,
 'LOG_LEVEL': 'INFO',
 'ROBOTSTXT_OBEY': Tr

CSV saved to:
- /Users/feddekoster/Desktop/ODM_Assignment2/product_observations_20260119_141105.csv
- /Users/feddekoster/Desktop/ODM_Assignment2/retailer_pages_20260119_141105.csv
Scrape finished. Outputs saved:
- /Users/feddekoster/Desktop/ODM_Assignment2/product_observations_20260119_141105.csv
- /Users/feddekoster/Desktop/ODM_Assignment2/retailer_pages_20260119_141105.csv
- /Users/feddekoster/Desktop/ODM_Assignment2/odm_competitor_benchmark_20260119_141105.sqlite


# 8. Post-run QA: Basic Data Quality Checks

We validate:
- counts per retailer,
- missing price ratios,
- basic discount sanity,
- status-code distribution.


In [12]:
# 8. Load and QA

import pandas as pd

df_prod = pd.read_csv(PROD_CSV_PATH) if PROD_CSV_PATH.exists() else pd.DataFrame()
df_ret  = pd.read_csv(RET_CSV_PATH)  if RET_CSV_PATH.exists()  else pd.DataFrame()

display(df_prod.head(10))
display(df_ret.head(10))

if not df_prod.empty:
    print("Rows:", len(df_prod))
    print("\nRows per retailer:")
    print(df_prod["retailer_key"].value_counts(dropna=False))

    print("\nHTTP status distribution:")
    print(df_prod["http_status"].value_counts(dropna=False).head(10))

    print("\nMissing price_current %:", round(df_prod["price_current"].isna().mean()*100, 2))
    print("Discount pct summary:")
    display(df_prod["discount_pct"].describe())

,retailer_key,category_key,product_url,page_title,retailer_product_id,brand,gtin_ean,price_current,price_reference,discount_pct,promo_flag,stock_text_raw,delivery_promise_text,observed_at,http_status,scrape_run_id
0,maxiaxi,microphones,https://www.maxiaxi.com/max-km01-karaoke-micro...,MAX KM01 Karaoke microfoon met speaker - Goud ...,130.139 130.139,MAX,8715693315042,13.9,17.95,22.56,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen bij je bezorgd! Bes...,2026-01-19T13:11:11.720276+00:00,200,20260119_141105
1,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-av510-bluetooth-...,Vonyx AV510 karaoke set met 2x microfoon kopen?,103.115 103.115,Vonyx,8715693319545,139.9,159.95,12.54,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:15.447600+00:00,200,20260119_141105
2,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-vmm100-podcast-s...,Vonyx VMM100 podcast set met mixer en accessoi...,60001097 60001097 173.403 172.638 188.020,Vonyx Vonyx Vonyx Vonyx,8720105710957 8715693293357 8715693334685 8715...,122.0,181.35,32.73,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:16.548452+00:00,200,20260119_141105
3,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-wat200-draadloze...,Vonyx WAT200 draadloze zender en ontvanger voo...,179.270 179.270,Vonyx,8715693346558,59.9,69.95,14.37,1,Op voorraad Op voorraad Op voorraad,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:17.940014+00:00,200,20260119_141105
4,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-dm57a-dynamische...,Vonyx DM57A Dynamische zang microfoon met kabe...,173.437 173.437,Vonyx,8715693283860,21.0,23.95,12.32,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen bij je bezorgd! Bes...,2026-01-19T13:11:19.407992+00:00,200,20260119_141105
5,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-draadloze-vhf-mi...,Vonyx STWM712 set 2-kanaals met 2 microfoons k...,179.183 179.183,Vonyx,8715693255423,67.0,77.95,14.05,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:22.244530+00:00,200,20260119_141105
6,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-wm82-draadloze-m...,Vonyx WM82 draadloze microfoonset met twee UHF...,179.214 179.214,Vonyx,8715693301441,115.0,134.95,14.78,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:24.049311+00:00,200,20260119_141105
7,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-hh12-handmicrofo...,Vonyx HH12 handmicrofoon voor Vonyx UHF system...,179.253 179.253,Vonyx,8715693303551,31.0,35.95,13.77,1,Op voorraad Op voorraad,Bestel en het wordt morgen bij je bezorgd! Bes...,2026-01-19T13:11:25.659262+00:00,200,20260119_141105
8,maxiaxi,microphones,https://www.maxiaxi.com/vonyx-hh10-handmicrofo...,Vonyx HH10 handmicrofoon voor Vonyx UHF system...,179.250 179.250,Vonyx,8715693303568,31.0,35.95,13.77,1,Op voorraad Op voorraad,Bestel en het wordt morgen bij je bezorgd! Bes...,2026-01-19T13:11:27.253268+00:00,200,20260119_141105
9,maxiaxi,microphones,https://www.maxiaxi.com/power-dynamics-pd504h-...,Power Dynamics PD504H microfoonsysteem met 4 m...,179.004 179.004,Power Dynamics,8715693301366,369.0,429.95,14.18,1,Op voorraad Op voorraad Op voorraad Op voorraa...,Bestel en het wordt morgen gratis bij je bezor...,2026-01-19T13:11:29.634023+00:00,200,20260119_141105


,retailer_key,page_type,source_url,captured_at,content_text,http_status,scrape_run_id
0,maxiaxi,shipping_returns,https://www.maxiaxi.com/klantenservice/,2026-01-19T13:11:07.476845+00:00,The store will not work correctly when cookies...,200,20260119_141105


Rows: 187

Rows per retailer:
retailer_key
maxiaxi    187
Name: count, dtype: int64

HTTP status distribution:
http_status
200    187
Name: count, dtype: int64

Missing price_current %: 0.0
Discount pct summary:


count    187.000000
mean      15.708770
std        6.688904
min        0.000000
25%       12.035000
50%       14.420000
75%       18.875000
max       37.750000
Name: discount_pct, dtype: float64

# 9. (Optional) Export Relational Tables to CSV for the Report Appendix

This is useful for a “deliverables bundle”:
- raw observations (time series),
- product_page registry,
- retailer page captures.


In [13]:
# 9. Export DB tables

with db_connect() as con:
    for table in ["retailer", "category", "product_page", "offer_observation", "retailer_page_capture"]:
        df = pd.read_sql_query(f"SELECT * FROM {table}", con)
        out = OUTPUT_DIR / f"{table}_{RUN_ID}.csv"
        df.to_csv(out, index=False)
        print("Exported:", out, "rows:", len(df))

Exported: /Users/feddekoster/Desktop/ODM_Assignment2/retailer_20260119_141105.csv rows: 1
Exported: /Users/feddekoster/Desktop/ODM_Assignment2/category_20260119_141105.csv rows: 1
Exported: /Users/feddekoster/Desktop/ODM_Assignment2/product_page_20260119_141105.csv rows: 187
Exported: /Users/feddekoster/Desktop/ODM_Assignment2/offer_observation_20260119_141105.csv rows: 187
Exported: /Users/feddekoster/Desktop/ODM_Assignment2/retailer_page_capture_20260119_141105.csv rows: 1
